In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

from urllib.parse import urlparse, parse_qs

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

In [2]:
list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：2



,No,Url
0,1,https://www.4s.com/en/ecatalog?part=Evaporator...
1,2,https://www.4s.com/en/ecatalog?part=Heater%20C...


In [3]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://www.4s.com/en/ecatalog?part=Evaporator...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)

list_dict_correct, page = [], 0
while True:

    count_retry = 0
    while True:
        count_retry += 1
        print(page + 1, count_retry)
        try:
            url_request = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start={page * 96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                list_dict = resp.json()

                if list_dict:
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = = = = = = = = = = = = = =

    if page == 0:
        count = list_dict[0]['maxRows']

    # = = = = = = = = = = = = = = =

    list_dict_correct.extend([{'No': dict_['rowId'],
                               'Part Number': dict_['basePart'],
                               'Url': f'''https://www.4s.com/en/ecatalog?part={dict_['webBase'].strip()}&type=p&search=s'''} for dict_ in list_dict])

    # = = = = = = = = = = = = = = =

    print(f' - {len(list_dict_correct)}/{count}')

    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if len(list_dict_correct) == count:
        break

list_dict_correct

1 1
 - 96/727
2 1
2 2
2 3
 - 192/727
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
 - 288/727
4 1
 - 384/727
5 1
 - 480/727
6 1
 - 576/727
7 1
7 2
 - 672/727
8 1
 - 727/727


[{'No': 1,
  'Part Number': '16149',
  'Url': 'https://www.4s.com/en/ecatalog?part=16149&type=p&search=s'},
 {'No': 2,
  'Part Number': '16152',
  'Url': 'https://www.4s.com/en/ecatalog?part=16152&type=p&search=s'},
 {'No': 3,
  'Part Number': '16155',
  'Url': 'https://www.4s.com/en/ecatalog?part=16155&type=p&search=s'},
 {'No': 4,
  'Part Number': '44001',
  'Url': 'https://www.4s.com/en/ecatalog?part=44001&type=p&search=s'},
 {'No': 5,
  'Part Number': '44002',
  'Url': 'https://www.4s.com/en/ecatalog?part=44002&type=p&search=s'},
 {'No': 6,
  'Part Number': '44003',
  'Url': 'https://www.4s.com/en/ecatalog?part=44003&type=p&search=s'},
 {'No': 7,
  'Part Number': '44014',
  'Url': 'https://www.4s.com/en/ecatalog?part=44014&type=p&search=s'},
 {'No': 8,
  'Part Number': '44015',
  'Url': 'https://www.4s.com/en/ecatalog?part=44015&type=p&search=s'},
 {'No': 9,
  'Part Number': '44017',
  'Url': 'https://www.4s.com/en/ecatalog?part=44017&type=p&search=s'},
 {'No': 10,
  'Part Number':

In [5]:
df_correct = pd.DataFrame(list_dict_correct)
df_correct.to_excel('./test_part_number.xlsx', index=False)

df_correct

,No,Part Number,Url
0,1,16149,https://www.4s.com/en/ecatalog?part=16149&type...
1,2,16152,https://www.4s.com/en/ecatalog?part=16152&type...
2,3,16155,https://www.4s.com/en/ecatalog?part=16155&type...
3,4,44001,https://www.4s.com/en/ecatalog?part=44001&type...
4,5,44002,https://www.4s.com/en/ecatalog?part=44002&type...
...,...,...,...
722,723,64181,https://www.4s.com/en/ecatalog?part=64181&type...
723,724,64182,https://www.4s.com/en/ecatalog?part=64182&type...
724,725,64183,https://www.4s.com/en/ecatalog?part=64183&type...
725,726,64184,https://www.4s.com/en/ecatalog?part=64184&type...


In [6]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.4s.com/en/ecatalog?part=Evaporator...
 Request_Url    https://ecatalog.smpcorp.com/V2/fs/api/part/pa...
 Name: 0, dtype: object]

In [7]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_number_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.4s.com/en/ecatalog?part=Evaporator...,https://ecatalog.smpcorp.com/V2/fs/api/part/pa...
